In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset = pd.get_dummies(dataset, drop_first=True)

In [5]:
dataset.head()

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1


In [11]:
iv=dataset[["Age","EstimatedSalary","Gender_Male"]]
dv=dataset["Purchased"]

In [8]:
iv.shape

(400, 3)

In [13]:
dv.shape

(400,)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iv, dv, test_size=1/3, random_state=0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
from sklearn.linear_model import LogisticRegression

In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg','lbfgs','liblinear','saga'],
             'penalty':['l2']}

grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=1, scoring='f1_weighted')

grid.fit(X_train, y_train)



Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ......penalty=l2, solver=newton-cg;, score=0.800 total time=   0.0s
[CV 2/5] END ......penalty=l2, solver=newton-cg;, score=0.811 total time=   0.0s
[CV 3/5] END ......penalty=l2, solver=newton-cg;, score=0.682 total time=   0.0s
[CV 4/5] END ......penalty=l2, solver=newton-cg;, score=0.924 total time=   0.0s
[CV 5/5] END ......penalty=l2, solver=newton-cg;, score=0.922 total time=   0.0s
[CV 1/5] END ..........penalty=l2, solver=lbfgs;, score=0.800 total time=   0.0s
[CV 2/5] END ..........penalty=l2, solver=lbfgs;, score=0.811 total time=   0.0s
[CV 3/5] END ..........penalty=l2, solver=lbfgs;, score=0.682 total time=   0.0s
[CV 4/5] END ..........penalty=l2, solver=lbfgs;, score=0.924 total time=   0.0s
[CV 5/5] END ..........penalty=l2, solver=lbfgs;, score=0.922 total time=   0.0s
[CV 1/5] END ......penalty=l2, solver=liblinear;, score=0.805 total time=   0.0s
[CV 2/5] END ......penalty=l2, solver=liblinear;,

GridSearchCV(estimator=LogisticRegression(), n_jobs=1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [21]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_predictions)

print(cm)

[[79  6]
 [10 39]]


In [22]:
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [23]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, grid_predictions, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'liblinear'}: 0.8794289739855382


In [24]:
print("The confusion Matrix:\n", cm)

The confusion Matrix:
 [[79  6]
 [10 39]]


In [25]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [27]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, grid.predict_proba(X_test)[:,1])

0.9507803121248499

In [28]:
table = pd.DataFrame.from_dict(re)

In [29]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011002,0.006100,0.002400,0.000489,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
1,0.005599,0.002727,0.002001,0.000634,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
2,0.002200,0.001470,0.001600,0.000490,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
3,0.002196,0.000390,0.001802,0.000401,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
